In [2]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

import joblib

from data_preprocessing import pipeline
from transformation import pipeline2

### 데이터 전처리

In [2]:
# 로우 데이터 불러오기
file_path = "./storage/raw_data/병합_청약매물_목록_정보_픽스.csv"
df = pd.read_csv(file_path, encoding='cp949')

# 데이터 전처리
preprocessing_pipeline =pipeline()
df = preprocessing_pipeline.transform(df)

# 파일 저장
file_version = "250317-01"
output_file = f"./storage/train_data/train-{file_version}.csv"
df.to_csv(output_file, index=False, encoding='cp949')

C:\Users\user\AppData\Local\Temp\ipykernel_9848\475702578.py:3: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='cp949')
c:\Users\user\developments\hk-project\src\data_preprocessing.py:136: DtypeWarning: Columns (36,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_estate_price = pd.read_csv(


In [71]:
df

,공고번호,주택명,공급지역코드,공급지역명,공급위치우편번호,공급위치,공급규모,모집공고일,청약접수시작일,청약접수종료일,...,미달여부,공급금액(최고가 기준),위도,경도,행정동코드,법정동코드,시도,시군구,읍면동1,읍면동2
0,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,Y,377000000.0,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동
1,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,N,377000000.0,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동
2,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,Y,377000000.0,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동
3,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,Y,377000000.0,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동
4,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,Y,377000000.0,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17245,2020000298,시흥장현 영무예다음,410,경기,14996,경기도 시흥장현 공공주택지구 B-9BL,747,2020-03-20,2020-03-30,2020-04-01,...,N,427200000.0,37.380161,126.790527,4.139063e+09,4.139013e+09,경기,시흥시,장곡동,장곡동
17246,2020000298,시흥장현 영무예다음,410,경기,14996,경기도 시흥장현 공공주택지구 B-9BL,747,2020-03-20,2020-03-30,2020-04-01,...,N,427200000.0,37.380161,126.790527,4.139063e+09,4.139013e+09,경기,시흥시,장곡동,장곡동
17247,2020000298,시흥장현 영무예다음,410,경기,14996,경기도 시흥장현 공공주택지구 B-9BL,747,2020-03-20,2020-03-30,2020-04-01,...,Y,427200000.0,37.380161,126.790527,4.139063e+09,4.139013e+09,경기,시흥시,장곡동,장곡동
17248,2020000298,시흥장현 영무예다음,410,경기,14996,경기도 시흥장현 공공주택지구 B-9BL,747,2020-03-20,2020-03-30,2020-04-01,...,Y,427200000.0,37.380161,126.790527,4.139063e+09,4.139013e+09,경기,시흥시,장곡동,장곡동


### 모델 학습 및 평가

In [1]:
# 전처리된 데이터 가져오기

# file_version = "250317-01"
# file_path = f"./storage/train_data/train-{file_version}.csv"
# df = pd.read_csv(file_path, encoding='cp949')

df = pd.read_csv("./datasets/train250317-01_topic_merged.csv")

NameError: name 'pd' is not defined

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17250 entries, 0 to 17249
Data columns (total 50 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   공고번호               17250 non-null  int64  
 1   주택명                17250 non-null  object 
 2   공급지역코드             17250 non-null  int64  
 3   공급지역명              17250 non-null  object 
 4   공급위치우편번호           17250 non-null  int64  
 5   공급위치               17250 non-null  object 
 6   공급규모               17250 non-null  int64  
 7   모집공고일              17250 non-null  object 
 8   청약접수시작일            17250 non-null  object 
 9   청약접수종료일            17250 non-null  object 
 10  당첨자발표일             17250 non-null  object 
 11  투기과열지구             17250 non-null  object 
 12  조정대상지역             17250 non-null  object 
 13  분양가상한제             17250 non-null  object 
 14  정비사업               17250 non-null  object 
 15  공공주택지구             17250 non-null  object 
 16  대규모택지개발지구          172

In [ ]:
X = df.drop(columns=["평균당첨가점"])
Y = df["평균당첨가점"]

X_train, X_test, y_train, y_test = train_test_split(
    X,  # 타겟(평균당첨가점) 제외
    Y,  # 타겟
    test_size=0.2,
    random_state=42,
    stratify=df["미달여부"]  # 분포 맞추기 위해 미달여부로 stratify
)

In [ ]:
feature_pipeline = pipeline2()

# 학습 데이터(X_train)에 fit
feature_pipeline.fit(X_train)

# 학습 데이터(X_train)를 변환
X_train_transformed = feature_pipeline.transform(X_train)

# 테스트 데이터(X_test)를 변환
X_test_transformed = feature_pipeline.transform(X_test)

In [6]:
# TODO: 여러 모델 학습 및 하이퍼파라미터 최적화

model = RandomForestRegressor(
    n_estimators=100,
    max_depth=None,
    random_state=42
)

# model.fit(x, y)

### 모델 저장 

In [7]:
# 모델 저장 및 로드

version = '0.0.1'

# 모델 저장
joblib.dump(model, f"./storage/trained_model/model_{version}.pkl")

['./storage/trained_model/model_0.0.1.pkl']

In [8]:
# 모델 로드
loaded_model = joblib.load("./storage/trained_model/model_0.0.1.pkl")

# 예측
# X_test = np.array([[1, 2]])
# prediction = loaded_model.predict(X_test)
# print("Prediction:", prediction)